In [ ]:
# prompt: import drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from glob import glob
from functools import reduce
from xml.etree import ElementTree as et

In [ ]:
# loading all xml files and storing it as List
xml_list = glob('/content/drive/MyDrive/VOC2007/Annotations/*.xml')

# Replacing \\ wiith / in Xml files
xml_list = [xml.replace('\\','/') for xml in xml_list]

In [ ]:
xml_list

In [ ]:
import xml.etree.ElementTree as et

def extract_data(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # Extracting FileName, Width, and Height of the image
    image_name = root.find('filename').text
    width = root.find('size/width').text
    height = root.find('size/height').text

    # Extracting bounding box coordinates and class names using XPath
    boxes = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        xmin = obj.find('bndbox/xmin').text
        xmax = obj.find('bndbox/xmax').text
        ymin = obj.find('bndbox/ymin').text
        ymax = obj.find('bndbox/ymax').text
        boxes.append([image_name, width, height, name, xmin, xmax, ymin, ymax])

    return boxes

bounding_boxes = []
for xml_file in xml_list:
    bounding_boxes.extend(extract_data(xml_file))

In [ ]:
bounding_boxes

In [ ]:
data = reduce(lambda x,y:x+y,bounding_boxes)
data

In [ ]:
# Reshape the flattened list into a list of tuples
bounding_boxes_list = [data[i:i+8] for i in range(0, len(data), 8)]

# Print the first few bounding box tuples for verification
for box in bounding_boxes_list[:5]:
    print(box)


['007970.jpg', '500', '375', 'car', '101', '326', '146', '299']
['007970.jpg', '500', '375', 'car', '296', '500', '163', '248']
['007970.jpg', '500', '375', 'person', '224', '263', '142', '184']
['007958.jpg', '500', '375', 'cat', '162', '330', '101', '306']
['007958.jpg', '500', '375', 'sofa', '1', '499', '1', '373']


In [ ]:
len(bounding_boxes_list)

15662

In [ ]:
df = pd.DataFrame(bounding_boxes_list,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [ ]:
df

,filename,width,height,name,xmin,xmax,ymin,ymax
0,007970.jpg,500,375,car,101,326,146,299
1,007970.jpg,500,375,car,296,500,163,248
2,007970.jpg,500,375,person,224,263,142,184
3,007958.jpg,500,375,cat,162,330,101,306
4,007958.jpg,500,375,sofa,1,499,1,373
...,...,...,...,...,...,...,...,...
15657,000021.jpg,336,500,dog,1,182,235,388
15658,000021.jpg,336,500,person,210,336,36,482
15659,000021.jpg,336,500,person,46,170,82,365
15660,000021.jpg,336,500,person,11,142,181,419


In [ ]:
df['name'].value_counts()

person         5447
car            1644
chair          1432
bottle          634
pottedplant     625
bird            599
dog             538
sofa            425
bicycle         418
horse           406
boat            398
motorbike       390
cat             389
tvmonitor       367
cow             356
sheep           353
aeroplane       331
train           328
diningtable     310
bus             272
Name: name, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15662 entries, 0 to 15661
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15662 non-null  object
 1   width     15662 non-null  int64 
 2   height    15662 non-null  int64 
 3   name      15662 non-null  object
 4   xmin      15662 non-null  int64 
 5   xmax      15662 non-null  int64 
 6   ymin      15662 non-null  int64 
 7   ymax      15662 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 979.0+ KB


In [ ]:
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15662 entries, 0 to 15661
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15662 non-null  object
 1   width     15662 non-null  int64 
 2   height    15662 non-null  int64 
 3   name      15662 non-null  object
 4   xmin      15662 non-null  int64 
 5   xmax      15662 non-null  int64 
 6   ymin      15662 non-null  int64 
 7   ymax      15662 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 979.0+ KB


In [ ]:
df['center_x']=((df['xmin']+df['xmax'])/2)/df['width']
df['center_y']=((df['ymin']+df['ymax'])/2)/df['height']
df['w']=(df['xmax']-df['xmin'])/df['width']
df['h']=(df['ymax']-df['ymin'])/df['height']
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,007970.jpg,500,375,car,101,326,146,299,0.427,0.593333,0.450,0.408000
1,007970.jpg,500,375,car,296,500,163,248,0.796,0.548000,0.408,0.226667
2,007970.jpg,500,375,person,224,263,142,184,0.487,0.434667,0.078,0.112000
3,007958.jpg,500,375,cat,162,330,101,306,0.492,0.542667,0.336,0.546667
4,007958.jpg,500,375,sofa,1,499,1,373,0.500,0.498667,0.996,0.992000


In [ ]:
class_id_map = {'person': 0, 'car': 1, 'chair': 2, 'bottle': 3,
                'pottedplant': 4, 'bird': 5, 'dog': 6,
                'sofa': 7, 'bicycle': 8, 'horse': 9, 'boat': 10,
                'motorbike': 11, 'cat': 12, 'tvmonitor': 13,
                'cow': 14, 'sheep': 15, 'aeroplane': 16,
                'train': 17, 'diningtable': 18, 'bus': 19}

# Add class_id column to DataFrame based on name
df['class_id'] = df['name'].map(class_id_map)

# Print DataFrame with class_id column added
print(df)

         filename  width  height    name  xmin  xmax  ymin  ymax  center_x  \
0      007970.jpg    500     375     car   101   326   146   299  0.427000   
1      007970.jpg    500     375     car   296   500   163   248  0.796000   
2      007970.jpg    500     375  person   224   263   142   184  0.487000   
3      007958.jpg    500     375     cat   162   330   101   306  0.492000   
4      007958.jpg    500     375    sofa     1   499     1   373  0.500000   
...           ...    ...     ...     ...   ...   ...   ...   ...       ...   
15657  000021.jpg    336     500     dog     1   182   235   388  0.272321   
15658  000021.jpg    336     500  person   210   336    36   482  0.812500   
15659  000021.jpg    336     500  person    46   170    82   365  0.321429   
15660  000021.jpg    336     500  person    11   142   181   419  0.227679   
15661  000020.jpg    375     500     car    33   371   148   416  0.538667   

       center_y         w         h  class_id  
0      0.593333

In [ ]:
df

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,class_id
0,007970.jpg,500,375,car,101,326,146,299,0.427000,0.593333,0.450000,0.408000,1
1,007970.jpg,500,375,car,296,500,163,248,0.796000,0.548000,0.408000,0.226667,1
2,007970.jpg,500,375,person,224,263,142,184,0.487000,0.434667,0.078000,0.112000,0
3,007958.jpg,500,375,cat,162,330,101,306,0.492000,0.542667,0.336000,0.546667,12
4,007958.jpg,500,375,sofa,1,499,1,373,0.500000,0.498667,0.996000,0.992000,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15657,000021.jpg,336,500,dog,1,182,235,388,0.272321,0.623000,0.538690,0.306000,6
15658,000021.jpg,336,500,person,210,336,36,482,0.812500,0.518000,0.375000,0.892000,0
15659,000021.jpg,336,500,person,46,170,82,365,0.321429,0.447000,0.369048,0.566000,0
15660,000021.jpg,336,500,person,11,142,181,419,0.227679,0.600000,0.389881,0.476000,0


In [ ]:
%cd /content/drive/MyDrive/VOC2007

/content/drive/MyDrive/VOC2007


In [ ]:
import os
import pandas as pd

# Path to the directory containing image files
directory = "/content/drive/MyDrive/VOC2007/JPEGImages"

# Get list of filenames in the directory
filenames = os.listdir(directory)

# Create DataFrame with filenames
img_df = pd.DataFrame(filenames, columns=['filename'])

# Sample 80% of the filenames for training
img_train = img_df.sample(frac=0.8, random_state=42)['filename'].tolist()

# Print the first few filenames in the training set for verification
print(img_train[:5])


['000298.jpg', '005521.jpg', '004656.jpg', '008607.jpg', '007754.jpg']


In [ ]:
img_train

In [ ]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])
img_test

In [ ]:
len(img_train),len(img_test)

(4017, 1004)

In [ ]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,class_id
3,007958.jpg,500,375,cat,162,330,101,306,0.492,0.542667,0.336,0.546667,12
4,007958.jpg,500,375,sofa,1,499,1,373,0.500,0.498667,0.996,0.992000,7
7,007963.jpg,500,375,person,285,304,291,337,0.589,0.837333,0.038,0.122667,0
8,007963.jpg,500,375,person,263,284,293,335,0.547,0.837333,0.042,0.112000,0
9,007963.jpg,500,375,car,457,500,230,248,0.957,0.637333,0.086,0.048000,1


In [ ]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,class_id
0,007970.jpg,500,375,car,101,326,146,299,0.427,0.593333,0.450,0.408000,1
1,007970.jpg,500,375,car,296,500,163,248,0.796,0.548000,0.408,0.226667,1
2,007970.jpg,500,375,person,224,263,142,184,0.487,0.434667,0.078,0.112000,0
5,007971.jpg,500,333,car,1,500,1,333,0.501,0.501502,0.998,0.996997,1
6,007971.jpg,500,333,person,179,323,83,261,0.502,0.516517,0.288,0.534535,0


In [ ]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,class_id
3,007958.jpg,500,375,cat,162,330,101,306,0.492,0.542667,0.336,0.546667,12
4,007958.jpg,500,375,sofa,1,499,1,373,0.500,0.498667,0.996,0.992000,7
7,007963.jpg,500,375,person,285,304,291,337,0.589,0.837333,0.038,0.122667,0
8,007963.jpg,500,375,person,263,284,293,335,0.547,0.837333,0.042,0.112000,0
9,007963.jpg,500,375,car,457,500,230,248,0.957,0.637333,0.086,0.048000,1


In [ ]:
import os
from shutil import move

In [ ]:
train_folder ='/content/drive/MyDrive/train_set'
test_folder ='/content/drive/MyDrive/test_set'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [ ]:
cols = ['class_id', 'center_x', 'center_y', 'w', 'h']

# Grouping train_df by the specified columns
groupby_obj_train = train_df.groupby(cols)

# Grouping test_df by the specified columns
groupby_obj_test = test_df.groupby(cols)

In [ ]:
import os

def generate_yolov8_txt(df, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for index, row in df.iterrows():
        filename = row['filename']
        basename = os.path.splitext(os.path.basename(filename))[0]
        txt_filename = os.path.join(output_dir, basename + '.txt')
        with open(txt_filename, 'w') as txt_file:
            for index, row in df[df['filename'] == filename].iterrows():
                class_id = row['class_id']
                center_x = row['center_x']
                center_y = row['center_y']
                width = row['w']
                height = row['h']
                # Write the annotation to the text file
                txt_file.write(f"{class_id} {center_x} {center_y} {width} {height}\n")

# Example usage for train_df
generate_yolov8_txt(train_df, "/content/drive/MyDrive/train_set")

# Example usage for test_df
generate_yolov8_txt(test_df, "/content/drive/MyDrive/test_set")


In [ ]:
!pip install yolov8 opencv torch

In [ ]:
!pip install --upgrade pip
!pip install opencv-python

In [ ]:
# prompt: check opencv installed or not

import cv2
print(cv2.__version__)


4.8.0


In [ ]:
!pip install torch

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import shutil
import os

# Source directory containing all images
source_directory = "/content/drive/MyDrive/VOC2007/JPEGImages"

# Destination directory where you want to copy the images
destination_directory = "/content/images"

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Loop through each row in the train_df DataFrame and copy the corresponding image
for index, row in train_df.iterrows():
    filename = row['filename']
    source_path = os.path.join(source_directory, filename)
    destination_path = os.path.join(destination_directory, filename)

    # Copy the image to the destination directory
    shutil.copy(source_path, destination_path)

print("Images copied to destination directory:", destination_directory)


Images copied to destination directory: /content/images


In [ ]:
import shutil
import os

# Source directory containing all images
source_directory = "/content/drive/MyDrive/VOC2007/JPEGImages"

# Destination directory where you want to copy the images
destination_directory = "/content/drive/MyDrive/test1/images"

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Loop through each row in the test_df DataFrame and copy the corresponding image
for index, row in test_df.iterrows():
    filename = row['filename']
    source_path = os.path.join(source_directory, filename)
    destination_path = os.path.join(destination_directory, filename)

    # Copy the image to the destination directory
    shutil.copy(source_path, destination_path)

print("Images copied to destination directory:", destination_directory)


Images copied to destination directory: /content/drive/MyDrive/test1/images


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='data.yaml', epochs=60, imgsz=640)

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=60, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

100%|██████████| 755k/755k [00:00<00:00, 16.2MB/s]


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/train1/labels.cache... 3360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3360/3360 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all       1001       3107      0.775       0.67      0.738      0.517

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      60/60      2.35G     0.7674     0.6119      1.051         40        640: 100%|██████████| 210/210 [01:22<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:12<00:00,  2.55it/s]


                   all       1001       3107      0.773      0.677      0.744      0.518

60 epochs completed in 1.677 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:15<00:00,  2.07it/s]


                   all       1001       3107      0.775      0.675      0.744      0.518
                person       1001       1168      0.843      0.761      0.834      0.547
                   car       1001        334      0.851      0.808      0.877      0.631
                 chair       1001        241      0.714      0.477      0.562      0.372
                bottle       1001        146      0.777      0.526      0.612      0.382
           pottedplant       1001         79      0.613      0.441      0.451      0.243
                  bird       1001        145      0.787      0.407      0.565      0.352
                   dog       1001        113      0.752      0.618       0.75       0.57
                  sofa       1001         87      0.688      0.563      0.613      0.402
               bicycle       1001         74      0.808      0.757       0.85      0.578
                 horse       1001         79      0.861       0.81      0.853      0.642
                  boa

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import math

# Replace with your actual YOLOv8 path
from ultralytics import YOLO

# Initialize webcam (adjust device index if needed)
cap = cv2.VideoCapture(1)
cap.set(3, 640)  # Set width
cap.set(4, 480)  # Set height

# Load trained model weights (update path if necessary)
model = YOLO("/content/drive/MyDrive/runs/detect/train5/weights/best.pt")

# Object class names (ensure correct ordering)
classNames = [
    "person", "car", "chair", "bottle", "pottedplant", "bird", "dog", "sofa",
    "bicycle", "horse", "boat", "motorbike", "cat", "tvmonitor", "cow", "sheep",
    "aeroplane", "train", "diningtable", "bus"
]

while True:
    ret, img = cap.read()
    if not ret:
        break  # Handle frame read errors

    # Perform object detection using YOLO
    results = model(img, stream=True)

    # Process each detected object
    for r in results:
        for box in r.pandas().xyxy[0]:  # Use 'xyxy[0]' for direct bounding box values
            x1, y1, x2, y2, conf, cls = box

            # Draw bounding box
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 255), 3)

            # Display confidence and class name
            conf = math.ceil((conf * 100)) / 100  # Adjust for clarity
            cv2.putText(img, f"{classNames[cls]} {conf:.2f}%", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display image with detections
    cv2_imshow(img)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
